# AstraForge Toolkit - Sandbox File Roundtrip

Create a sandbox session, upload a file, and read it back without using DeepAgent.

Prereqs:
- Backend running at `http://localhost:8001` (e.g., `make backend-serve`)
- An API key (create via the in-app API Keys screen or `/api/api-keys/`)
- Package installed (`pip install astraforge-toolkit` or from PyPI)


## Notebook walk-through

1. Install the toolkit and dotenv helper so environment variables can be resolved within `examples/`.
2. Load the `.env` file, derive `BASE_URL`/`API_KEY`, and create a `DeepAgentClient` plus sandbox session to operate within `/workspace`.
3. The notebook uploads a simple text file, then immediately reads `client.get_file_content` to verify the same content returns, demonstrating sandbox CRUD without DeepAgent.
4. This pattern is handy for seeding files that later DeepAgent agents can inspect, and you can add a manual cleanup call (`client.stop_sandbox_session`) once you finish experimenting.


### Guided tutorial notes

- Run the install + dotenv cells once per session to refresh environment variables.
- The sandbox session information shows where `/workspace` lives so you can upload other artifacts.
- Uploading and immediate reading proves the backend’s file CRUD, so extend with other file types if desired.
- Stop the sandbox session once you finish experimenting to avoid leftover resources.


In [1]:
# Install the package + dotenv (run from the examples/ folder)
%pip install astraforge-toolkit python-dotenv --quiet


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import os

BASE_URL = os.getenv("ASTRA_FORGE_API_URL", "http://localhost:8001/api")
API_KEY = os.getenv("ASTRA_FORGE_API_KEY", "") # for local setup go to http://localhost:5174/app/api-keys

if not API_KEY:
    raise RuntimeError("Set ASTRA_FORGE_API_KEY in your environment")

In [4]:
from astraforge_toolkit import DeepAgentClient

client = DeepAgentClient(base_url=BASE_URL, api_key=API_KEY)
sandbox_session = client.create_sandbox_session()
SANDBOX_SESSION_ID = sandbox_session.session_id
print(f"Sandbox session: {SANDBOX_SESSION_ID} (workspace: {sandbox_session.workspace_path})")

Sandbox session: 0ea65c77-c4d6-4098-95ed-ea5fb59affa3 (workspace: /workspace)


In [5]:
path = "/workspace/hello.txt"
client.upload_file(SANDBOX_SESSION_ID, path, content="Hello from AstraForge!\n")
client.get_file_content(SANDBOX_SESSION_ID, path, encoding="utf-8")

'Hello from AstraForge!\n'

In [6]:
# Stop the session when you are done.
client.stop_sandbox_session(SANDBOX_SESSION_ID)